In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# subworkflow to convert all alto files in the subdirectories of one directory into text files
import nlppln

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir='../ochre/cwl/')
    print(wf.list_steps())
    
    in_dir = wf.add_input(in_dir='Directory')
    
    # inputs with default values
    recursive = wf.add_input(recursive='boolean', default=True)
    endswith = wf.add_input(endswith='string', default='alto.xml')
    element = wf.add_input(element='string[]', default=['SP'])
    in_fmt = wf.add_input(in_fmt='string', default='alto')
    out_fmt = wf.add_input(out_fmt='string', default='text')
    
    in_files = wf.ls(in_dir=in_dir, recursive=recursive, endswith=endswith)
    cleaned_files = wf.remove_xml_elements(element=element, xml_file=in_files, 
                                          scatter='xml_file', scatter_method='dotproduct')
    text_files = wf.ocr_transform(in_file=cleaned_files, in_fmt=in_fmt, out_fmt=out_fmt,
                                  scatter='in_file', scatter_method='dotproduct')
    
    #wf.add_outputs(xml=cleaned_files)
    wf.add_outputs(text_files=text_files)
    
    wf.save('../ochre/cwl/kb-tss-preprocess-single-dir.cwl', wd=True)

In [ ]:
# Workflow to preprocess all kb_tss data
import nlppln

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir='../ochre/cwl/')
    print(wf.list_steps())
    
    karmac_dir = wf.add_input(karmac_dir='Directory')
    original_dir = wf.add_input(orignal_dir='Directory')
    xcago_dir = wf.add_input(xcago_dir='Directory')
    
    karmac_name = wf.add_input(karmac_name='string', default='Karmac')
    original_name = wf.add_input(original_name='string', default='Origineel')
    xcago_name = wf.add_input(xcago_name='string', default='X-Cago')
    
    karmac_aligned_name = wf.add_input(karmac_name='string', default='align-Karmac-Origineel')
    xcago_aligned_name = wf.add_input(xcago_name='string', default='aign-X-Cago-Origineel')
    
    karmac_files_with_empty = wf.kb_tss_preprocess_single_dir(in_dir=karmac_dir)
    original_files_with_empty = wf.kb_tss_preprocess_single_dir(in_dir=original_dir)
    xcago_files_with_empty = wf.kb_tss_preprocess_single_dir(in_dir=xcago_dir)
    
    karmac_dir_with_empty = wf.save_files_to_dir(dir_name=karmac_name, in_files=karmac_files_with_empty)
    original_dir_with_empty = wf.save_files_to_dir(dir_name=original_name, in_files=original_files_with_empty)
    xcago_dir_with_empty = wf.save_files_to_dir(dir_name=xcago_name, in_files=xcago_files_with_empty)
    
    karmac_files, original_files, xcago_files = wf.kb_tss_remove_empty_files(in_dir1=karmac_dir_with_empty, 
                                                                             in_dir2=original_dir_with_empty,
                                                                             in_dir3=xcago_dir_with_empty)
    
    karmac_dir_new = wf.save_files_to_dir(dir_name=karmac_name, in_files=karmac_files)
    original_dir_new = wf.save_files_to_dir(dir_name=original_name, in_files=original_files)
    xcago_dir_new = wf.save_files_to_dir(dir_name=xcago_name, in_files=xcago_files)
    
    karmac_alignments, karmac_changes, karmac_metadata = wf.align_texts_wf(gs=karmac_files, ocr=original_files)
    xcago_alignments, xcago_changes, xcago_metadata = wf.align_texts_wf(gs=xcago_files, ocr=original_files)
    
    karmac_align_dir = wf.save_files_to_dir(dir_name=karmac_aligned_name, in_files=karmac_alignments)
    xcago_align_dir = wf.save_files_to_dir(dir_name=xcago_aligned_name, in_files=xcago_alignments)
    
    wf.add_outputs(karmac=karmac_dir_new)
    wf.add_outputs(original=original_dir_new)
    wf.add_outputs(xcago=xcago_dir_new)
    wf.add_outputs(karmac_align=karmac_align_dir)
    wf.add_outputs(xcago_align=xcago_align_dir)
    
    wf.save('../ochre/cwl/kb-tss-preprocess-all.cwl', pack=True)